In [1]:
import sage_data_client
import matplotlib.pyplot as plt
import pandas as pd
from metpy.calc import dewpoint_from_relative_humidity, wet_bulb_temperature
from metpy.units import units
from PIL import Image
import numpy as np
import datetime
import xarray as xr

from matplotlib.dates import DateFormatter

In [3]:
st = datetime.datetime(2023,6,6)
   

In [28]:
def ingest_aqt(st):
    hours = 24
    start = st.strftime('%Y-%m-%dT%H:%M:%SZ')
    end = (st + datetime.timedelta(hours=hours)).strftime('%Y-%m-%dT%H:%M:%SZ')

    df_aq = sage_data_client.query(
        start=start,
        end=end, 
        filter={
            "plugin": "registry.sagecontinuum.org/jrobrien/waggle-aqt:0.23.5.04.*",
            "vsn": "W08D"
        }
    )

    pm25 = df_aq[df_aq['name']=='aqt.particle.pm2.5']
    pm10 = df_aq[df_aq['name']=='aqt.particle.pm1']
    pm100 = df_aq[df_aq['name']=='aqt.particle.pm10']

    no = df_aq[df_aq['name']=='aqt.gas.no']
    o3 = df_aq[df_aq['name']=='aqt.gas.ozone']
    no2 = df_aq[df_aq['name']=='aqt.gas.no2']
    co = df_aq[df_aq['name']=='aqt.gas.co']
    aqtemp = df_aq[df_aq['name']=='aqt.env.temp']
    aqhum = df_aq[df_aq['name']=='aqt.env.humidity']


    pm25['time'] = pd.DatetimeIndex(pm25['timestamp'].values)

    aqvals = pm25.set_index('time')
    aqvals['pm2.5'] = aqvals.value.to_numpy().astype(float)
    aqvals['pm1.0'] = pm10.value.to_numpy().astype(float)
    aqvals['pm10.0'] = pm100.value.to_numpy().astype(float)

    aqvals['no'] = no.value.to_numpy().astype(float)
    aqvals['o3'] = o3.value.to_numpy().astype(float)
    aqvals['no2'] = no2.value.to_numpy().astype(float)
    aqvals['co'] = co.value.to_numpy().astype(float)
    aqvals['temp'] =  aqtemp.value.to_numpy().astype(float)
    aqvals['hum'] =  aqhum.value.to_numpy().astype(float)


    dp = dewpoint_from_relative_humidity( aqvals.temp.to_numpy() * units.degC, 
                                         aqvals.hum.to_numpy() * units.percent)

    aqvals['dewpoint'] = dp

    _ = aqvals.pop('value')
    _ = aqvals.pop('timestamp')
    
    
    fname = st.strftime('/Users/scollis/data/aqt/CMS_aqt580_NEIU_a1_%Y%m%d_%H%M%SZ.nc')
    valsxr = xr.Dataset.from_dataframe(aqvals)
    if valsxr['pm2.5'].shape[0] > 0:
        valsxr.to_netcdf(fname)
    else:
        print('not saving... no data')



In [65]:
ingest_aqt(datetime.datetime(2023,6,11))

/var/folders/f4/f3jp1t2x0hddhf4vz50gzlnc0000gn/T/ipykernel_98993/4276351945.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pm25['time'] = pd.DatetimeIndex(pm25['timestamp'].values)


In [27]:
valsxr['pm2.5'].shape[0] > 0

True

In [64]:
! rm /Users/scollis/data/aqt/CMS_aqt580_NEIU_a1_20230611_000000Z.nc